- pogledati kako normalization layer radi
- dodati sentance embedding
- runat na cudi
- dodati loader

In [2]:
import torch
import torch.nn as nn
import math

#### MultiHadAttention

Attention($Q$, $K$, $V$) = softmax($\frac {QK^T} {\sqrt {d_k}}$)$V$

![attention](./attention.jpg)

In [40]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.output_linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # matmul Q and K and scale
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # apply mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # softmax layer
        attention = torch.softmax(scores, dim=-1)

        # matmul attention and V
        context = torch.matmul(attention, V)

        return context
    
    # dijeli x na batch_size, broj glava, duljinu rijeci i d_k
    def split_heads(self, x):
        batch_size, seq_len, _ = x.size()

        return x.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        batch_size, _, seq_len, _ = x.size()

        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
    
    def forward(self, Q, K, V, mask=None):
        # batch_size x seq_len x d_model
        Q = self.W_Q(Q)
        K = self.W_K(K)
        V = self.W_V(V)

        # splitaj Q, K i V na broj glava
        # batch_size x num_heads x seq_len x d_k
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)

        attention = self.scaled_dot_product_attention(Q, K, V, mask)

        # vrati u oblik prije splitanja
        # batch_size x seq_len x d_model
        attention = self.combine_heads(attention)

        output = self.output_linear(attention)

        return output


#### Positional Encoding

$PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})$ za parne

$PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})$ za neparne

odnosno

$PE_{(pos, i)} = sin(pos / 10000^{i / d_{model}})$ za parne

$PE_{(pos, i+1)} = cos(pos / 10000^{i / d_{model}})$ za neparne

In [41]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self, x):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_pe = torch.sin(position / denominator)
        odd_pe = torch.cos(position / denominator)
        stacked = torch.stack([even_pe, odd_pe], dim=2)
        pe = torch.flatten(stacked, start_dim=1, end_dim=2)
        
        return x + pe

#### Position wise feed forward network

Sluzi za dodatno ucenje

In [42]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()

        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear1(x) # d_model x d_ff
        x = torch.relu(x)
        x = self.linear2(x) # d_ff x d_model

        return x

#### Encoder

![encoder](./encoder.jpg)

In [43]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()

        self.multi_head_attention = MultiHeadAttention(d_model, num_heads)
        self.position_wise_feed_forward = PositionWiseFeedForward(d_model, d_ff)

        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # multi-head attention
        attention = self.multi_head_attention(x, x, x, mask)

        # add and norm
        x = self.layer_norm1(x + attention)
        x = self.dropout1(x)

        # position-wise feed forward
        feed_forward = self.position_wise_feed_forward(x)

        # add and norm
        x = self.layer_norm2(x + feed_forward)
        x = self.dropout2(x)

        return x

#### Decoder

![decoder](./decoder.jpg)

In [44]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super().__init__()

        self.multi_head_attention1 = MultiHeadAttention(d_model, num_heads)
        self.multi_head_attention2 = MultiHeadAttention(d_model, num_heads)

        self.position_wise_feed_forward = PositionWiseFeedForward(d_model, d_ff)

        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.layer_norm3 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, trg_mask=None):
        # masked multi-head attention
        masked_attention = self.multi_head_attention1(x, x, x, trg_mask)

        # add and norm
        x = self.layer_norm1(x + masked_attention)
        x = self.dropout1(x)

        # multi-head attention
        attention = self.multi_head_attention2(x, encoder_output, encoder_output, src_mask)

        # add and norm
        x = self.layer_norm2(x + attention)
        x = self.dropout2(x)

        # position-wise feed forward
        feed_forward = self.position_wise_feed_forward(x)

        # add and norm
        x = self.layer_norm3(x + feed_forward)
        x = self.dropout3(x)

        return x
        

#### Transformer

In [45]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_lenth, dropout):
        super().__init__()

        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.positional_encoding = PositionalEncoding(d_model, max_seq_lenth)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.linear = nn.Linear(d_model, tgt_vocab_size)
    
    # def generate_mask(self, src, tgt):
    #     src_mask = (src != 0).unsqueeze(-2)

    #     tgt_mask = (tgt != 0).unsqueeze(-2)
    #     tgt_mask = tgt_mask & torch.tril(torch.ones(tgt.size(-1), tgt.size(-1))).type_as(tgt_mask).unsqueeze(0)

    #     return src_mask, tgt_mask

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        
        return src_mask, tgt_mask
    
    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        src = self.src_embedding(src)
        tgt = self.tgt_embedding(tgt)

        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)

        for layer in self.encoder_layers:
            src = layer(src, src_mask)
        
        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)
        
        output = self.linear(tgt)

        return output

#### Initialization

In [46]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

In [47]:
val_src_data = torch.randn(1, 100, 512)
val_src_data.size()

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

#### Loading data

In [48]:
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

#### Training

In [49]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()

    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.703485488891602
Epoch: 2, Loss: 8.652273178100586
Epoch: 3, Loss: 8.610093116760254
Epoch: 4, Loss: 8.581259727478027
Epoch: 5, Loss: 8.549642562866211
Epoch: 6, Loss: 8.524422645568848
Epoch: 7, Loss: 8.500144004821777
Epoch: 8, Loss: 8.48416519165039
Epoch: 9, Loss: 8.46324348449707
Epoch: 10, Loss: 8.438236236572266
Epoch: 11, Loss: 8.420746803283691
Epoch: 12, Loss: 8.39939022064209
Epoch: 13, Loss: 8.377776145935059
Epoch: 14, Loss: 8.369438171386719
Epoch: 15, Loss: 8.344329833984375
Epoch: 16, Loss: 8.33425235748291
Epoch: 17, Loss: 8.312045097351074
Epoch: 18, Loss: 8.300150871276855
Epoch: 19, Loss: 8.26464557647705
Epoch: 20, Loss: 8.256332397460938
Epoch: 21, Loss: 8.234614372253418
Epoch: 22, Loss: 8.219710350036621
Epoch: 23, Loss: 8.200614929199219
Epoch: 24, Loss: 8.188332557678223
Epoch: 25, Loss: 8.170958518981934
Epoch: 26, Loss: 8.14598274230957
Epoch: 27, Loss: 8.135903358459473
Epoch: 28, Loss: 8.123311042785645
Epoch: 29, Loss: 8.1107816696167
Ep

In [50]:
W_Q = nn.Linear(512, 512)

def split_heads(x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, 8, 64).transpose(1, 2)

split_heads(W_Q(val_src_data)).size()

torch.Size([1, 8, 100, 64])